# Topic Modeling and Gibbs Sampling

Задача: описать текст через распределение весов по некоторому фиксированному набору топиков (тегов). Например, для набора тегов Политика, Военные сражения, Спорт, Интернет, Драма представить роман "Война и мир" как вектор (0.3, 0.2, 0, 0, 0.5), а статью в газете про допинг в велоспорте как вектор (0.1, 0, 0.7, 0, 0.2).

Для чего, например, это нужно: имея векторное представление для текстов, тексты можно сравнивать, рекомендовать похожие.

Условие: даны только набор текстов и количество тем.

## Немного теории

Будем представлять текст как неупорядоченный набор слов (Bag-of-words model). Предположим, что имеется K тегов и для каждого тега выбрано распределение $\phi_k$ над списком всевозможных слов (словарем из N слов). По сути, каждое $\phi_k$ - это вектор длины N из неотрицательных величин, в сумме дающих 1. Вектора $\phi_k$ независимы и моделируются распредеделением Дирихле $Dir(\beta)$. Теперь, чтобы собрать текст d из $n$ слов, будем действовать по следующей схеме:


* выберем распредление для тегов $\theta_d$. Вновь, $\theta_d$ - это вектор длины K из неотрицательных величин, в сумме дающих 1. Поэтому естественно брать $\theta_d \sim Dir(\theta | \alpha)$

* Для i от 1 до n:
  * выберем тег $z_i$ согласно распределению $\theta_d$
  * выберем слово $w_i$ из распределения для данного тега, т.е. $w_i \sim \phi_{z_i}$
  * добавляем слово $w_i$ в текст.

Полученная модель называется моделью LDA (Latent Dirichlet Allocation). Описанная  схема задает совместное распределение скрытых и наблюдаемых параметров по всем текстам корпуса размера M в виде:


$p(\textbf{w}, \textbf{z}, \theta, \phi | \alpha, \beta) = Dir(\theta | \alpha) Dir(\phi|\beta)Cat(\textbf{z}|\theta)Cat(\textbf{w}|\phi_z)$.

Здесь $\textbf{w}$ и $\textbf{z}$ обозначают вектора слов и тегов по всем текстам, $\theta$ - набор из $\theta_d$ для каждого документа (матрица $M\times K$), $\phi$ - набор из $\phi_k$ для каждого тега (матрица $K\times N$).


![img](https://www.researchgate.net/publication/336065245/figure/fig1/AS:807371718815752@1569503826964/Latent-Dirichlet-allocation-LDA-process-and-its-two-outputs-a-LDA-document.ppm)

Наша задача - восстановить распределение $p(\textbf{z}, \theta, \phi | \textbf{w}, \alpha, \beta)$.


Немного упростим жизнь, и поставим себе задачей восстановить распределение $p(\textbf{z} | \textbf{w}, \alpha, \beta) = \int\int p(\textbf{z}, \theta, \phi | \textbf{w}, \alpha, \beta)\textrm{d}\theta\textrm{d}\phi$.

В этот момент на помощь приходить алгоритм Gibbs Sampling. Напомним, для оценки набора парамеров $\textbf{z} = (z_1, z_2, ..., z_m)$ используется схема:

$z_i^{(t)} \sim p(z_i^{(t)}\ | \ z_1=z_1^{t}, ..., z_{i-1}=z_{i-1}^{t},
z_{i+1}=z_{i+1}^{t-1}, z_{m}=z_{m}^{t-1})$.

Условные распределения выводятся так. Сначала замечаем, что

$p(z_i|\textbf{z}_{\hat{i}}, \textbf{w}, \alpha, \beta) = \frac{p(z_i,\textbf{z}_{\hat{i}}, \textbf{w}| \alpha, \beta)}{p(\textbf{z}_{\hat{i}}, \textbf{w}| \alpha, \beta)}  = \frac{p(\textbf{z}, \textbf{w}| \alpha, \beta)}{p(\textbf{z}_{\hat{i}}, \textbf{w}_{\hat{i}}| \alpha, \beta)  p(w_i|\alpha, \beta)}$.

Здесь $\textbf{z}_{\hat{i}}$, $\textbf{w}_{\hat{i}}$ - вектора без $i$-oй копмоненты. 

Далее расписываем:

$p(\textbf{z}, \textbf{w}| \alpha, \beta) = \int\int p(\textbf{z}, \textbf{w}, \theta, \phi| \alpha, \beta)\textrm{d}\theta\textrm{d}\phi = \int\int Dir(\theta | \alpha) Dir(\phi|\beta)Cat(\textbf{z}|\theta)Cat(\textbf{w}|\phi_z)\textrm{d}\theta\textrm{d}\phi = 
\int Dir(\theta | \alpha) Cat(\textbf{z}|\theta)\textrm{d}\theta \int Dir(\phi|\beta)Cat(\textbf{w}|\phi_z)\textrm{d}\phi$

и обнаруживаем, что оба интеграла в последнем выражении вычисляются аналитически. Для примера первый:

$\int Dir(\theta | \alpha) Cat(\textbf{z}|\theta)\textrm{d}\theta = \prod\limits_d \int Dir(\theta_d | \alpha) Cat(\textbf{z}_d|\theta_d)\textrm{d}\theta_d = \prod\limits_d \int \frac{1}{B(\alpha)}\prod\limits_k \theta_{d, k}^{\alpha-1}\prod\limits_i \theta_{d, z_i}\textrm{d}\theta_d = \prod\limits_d\frac{1}{B(\alpha)}\int\prod\limits_k \theta_{d, k}^{n_{d, k} + \alpha - 1}\textrm{d}\theta_d = \prod\limits_d \frac{B(n_{d,\cdot} + \alpha)}{B(\alpha)}$.

Здесь  $n_{d,k}$ - количество тэгов $k$ в тексте $d$, $n_{d,\cdot}$ - вектор длины $K$ из этих величин.

Аналогично, второй интеграл 
$\int Dir(\phi|\beta)Cat(\textbf{w}|\phi_z)\textrm{d}\phi = \prod\limits_k \frac{B(n_{k,\cdot} + \beta)}{B(\beta)}$,

где $n_{k,\cdot}$ - вектор длины $N$ встречаемости слов внутри тэга $k$.

Получаем: 

$p(\textbf{z}, \textbf{w}| \alpha, \beta) = \prod\limits_d \frac{B(n_{d,\cdot} + \alpha)}{B(\alpha)} \prod\limits_k \frac{B(n_{k,\cdot} + \beta)}{B(\beta)}$.

Теперь
$p(z_i|\textbf{z}_{\hat{i}}, \textbf{w}, \alpha, \beta) \propto \prod\limits_d \frac{B(n_{d,\cdot} + \alpha)}{B(n_{d,\cdot}^{\hat{i}} + \alpha)} \prod\limits_k \frac{B(n_{k,\cdot} + \beta)}{B(n_{k,\cdot}^{\hat{i}} + \beta)}$.

Знак $\propto$ означает пропорциональность с точностью до общего множителя $p(w_i|\alpha, \beta)$. Векторы $n_{d,\cdot}^{\hat{i}}$ и $n_{k,\cdot}^{\hat{i}}$ получены из векторов $n_{d,\cdot}$ и $n_{k,\cdot}$ после выбрасывания $z_i$. 

Выражение упрощается дальше, расписывая бета-функцию через гамма-функции. Напомним,
$B(x_1, ..., x_m) = \frac{\Gamma(x_1)\cdot...\cdot\Gamma(x_m)}{\Gamma(x_1 + ... + x_m)}$, а также $\Gamma(n) = (n-1)\Gamma(n-1)$. Получим:

$p(z_i=k |\textbf{z}_{\hat{i}}, \textbf{w}, \alpha, \beta) \propto (n_{d_i, k}^{\hat{i}} + \alpha_k) \frac{n_{k, w_i}^{\hat{i}} + \beta_{w_i}}{\sum\limits_{w}(n_{k, w}^{\hat{i}} + \beta_{w})}$.


С этого места можно полностью собрать алгоритм моделирования плотности $p(\textbf{z}| \textbf{w}, \alpha, \beta)$. Введем  обозначение $n_k$ - количество слов, отнесенных к тегу $k$, $W$ - общее количество слов в корпусе, $\beta_{sum} = \sum\limits_w\beta_w$

Алгоритм:

* заведем счетчики $n_{k, w}$, $n_{d, k}$, $n_k$
* случайным образом расставим теги словам, обновим счетчики $n_{k, w}$, $n_{d, k}$, $n_k$
* пока не сойдемся к стационарному режиму:
  * для каждого $i$ от 1 до $W$:
      * для каждого $k$ от 1 до $K$:
        * $I = I\{z_i = k\}$ (индикатор)
        * вычисляем $p_k = (n_{d_i, k} + \alpha_k - I) \frac{n_{k, w_i} + \beta_{w_i} - I}{n_k + \beta_{sum} - I}$
      * сэмплим новый $z_i$ из полученного распределения $(p_1, ..., p_K)$
      * обновляем счетчики для учета обновленого значения $z_i$

На практике удобно реализовавать так:

* заведем счетчики $n_{k, w}$, $n_{d, k}$, $n_k$, заполненные нулями
* случайным образом расставим теги словам, обновим счетчики $n_{k, w}$, $n_{d, k}$, $n_k$
* пока не сойдемся к стационарному режиму:
  * для каждого $i$ от 1 до $W$:
      * $n_{d_i, z_i} \mathrel{-}= 1$, $n_{z_i, w_i} \mathrel{-}= 1$, $n_{z_i} \mathrel{-}= 1$
      * для каждого $k$ от 1 до $K$:
        * вычисляем $p_k = (n_{d, k} + \alpha_k) \frac{n_{k, w_i} + \beta_{w_i}}{n_k + \beta_{sum}}$
      * сэмплим новый $z_i$ из полученного распределения $(p_1, ..., p_K)$
      * $n_{d_i, z_i} \mathrel{+}= 1$, $n_{z_i, w_i} \mathrel{+}= 1$, $n_{z_i} \mathrel{+}= 1$





Восстановив распредление для $\textbf{z}$, можем оценить $\theta$ и $\phi$, о которых мы ненадолго забыли. Оценить можно, например, через матожидание по апостериорным распределениям. Получите формулы самостоятельно!

Литература:

http://u.cs.biu.ac.il/~89-680/darling-lda.pdf

https://www.cs.cmu.edu/~mgormley/courses/10701-f16/slides/lecture20-topic-models.pdf

Перейдем к практике.

## Датасет

Возьмем популярный датасет [20 Newsgroups](http://qwone.com/~jason/20Newsgroups/), встроенный в пакет ```sklearn```. Датасет состоит из ~20К текстов, классифицированных на 20 категорий. Датасет разбит на ```train``` и ```test```. Для загрузки используем  модуль ```fetch_20newsgroups```, в параметрах указать, что мета информацию о тексте загружать не нужно:

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

Выведем список категорий текстов:

In [2]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Атрибут ```traget``` хранит номера категорий для текстов из обучающей выборки:

In [3]:
newsgroups_train.target[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

Доступ к самим текстам через атрибут ```data```. Выведем текст и категорию случайного примера из обучающего датасета:

In [4]:
n = 854
print('Topic = {0}\n'.format(newsgroups_train.target_names[newsgroups_train.target[n]]))
print(newsgroups_train.data[n])

Topic = rec.motorcycles

hey... I'm pretty new to the wonderful world of motorcycles... I just
bought
a used 81 Kaw KZ650 CSR from a friend.... I was just wondering what kind of

saddle bags I could get for it (since I know nothing about them)  are there
bags for the gas tank?  how much would some cost, and how much do they
hold?
thanks for your advice!!!  I may be new to riding, but I love it
already!!!!
:)




## Векторное представление текста

Представим текст как вектор индикаторов вхождений слов из некоторого словаря в текст. Это простейшая модель BOF. 

Сформируем словарь на основе нашего набора текстов. Для этого используем модуль ```CountVectorizer```:

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,
                             analyzer='word', binary=True,min_df = 8,max_df = 0.09)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.09, max_features=None, min_df=8,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=frozenset({'she', 'cry', 'hereupon', 'everyone', 'themselves', 'between', 'also', 'for', 'who', 'there', 'eg', 'seeming', 'or', 'so', 'beside', 'the', 'hundred', 'describe', 'see', 'we', 'few', 'same', 'otherwise', 'at', 'interest', 'may', 'every', 'namely', 'among', 'how', 'everywhere', ...'elsewhere', 'him', 'thence', 'bottom', 'nevertheless', 'from', 'another', 'sometime', 'each', 'a'}),
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

Количество проиндексированных слов:

In [6]:
len(vectorizer.vocabulary_)

12424

Проиндексированные слова и их индексы:

In [7]:
vectorizer.vocabulary_

{'wondering': 12203,
 'enlighten': 4343,
 'car': 2383,
 'saw': 9905,
 'day': 3498,
 'door': 3999,
 'sports': 10570,
 'looked': 6919,
 'late': 6659,
 '60s': 694,
 'early': 4135,
 '70s': 766,
 'called': 2331,
 'doors': 4000,
 'small': 10397,
 'addition': 1082,
 'bumper': 2246,
 'separate': 10091,
 'rest': 9555,
 'body': 2052,
 'model': 7438,
 'engine': 4327,
 'specs': 10528,
 'years': 12355,
 'production': 8836,
 'history': 5640,
 'info': 6041,
 'looking': 6920,
 'mail': 7041,
 'fair': 4674,
 'number': 7849,
 'brave': 2144,
 'souls': 10480,
 'upgraded': 11710,
 'si': 10257,
 'clock': 2717,
 'oscillator': 8083,
 'shared': 10169,
 'experiences': 4563,
 'poll': 8586,
 'send': 10071,
 'brief': 2171,
 'message': 7287,
 'detailing': 3718,
 'procedure': 8814,
 'speed': 10534,
 'cpu': 3290,
 'rated': 9181,
 'add': 1078,
 'cards': 2391,
 'adapters': 1075,
 'heat': 5559,
 'sinks': 10318,
 'hour': 5734,
 'usage': 11734,
 'floppy': 4897,
 'disk': 3889,
 'functionality': 5079,
 '800': 805,
 'floppies

Индекс, например, для слова anyone:

In [8]:
vectorizer.vocabulary_.get('car')

2383

А теперь преобразуем строку в вектор:

In [9]:
text = 'I was wondering if anyone out there could enlighten me on this car I saw'
x = vectorizer.transform([text])

Какой тип имеет объект, на который указывает ```x```?

In [10]:
type(x)

scipy.sparse.csr.csr_matrix

Разреженная матрица!

### Отступление про разреженные матрицы

Список ненулевых элементов матрицы:

In [11]:
x.data

array([1, 1, 1, 1], dtype=int64)

Индексы строк и столбцов для ненулевых элементов:

In [12]:
x.nonzero()

(array([0, 0, 0, 0], dtype=int32),
 array([ 2383,  4343,  9905, 12203], dtype=int32))

Преобразование к объекту ndarray (именно после приведения к такому виду разреженные матрицы можно подставлять в функции, например, библиотеки Numpy):

In [13]:
x.toarray()

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

Вернемся к словарю. Раскодируем вектор ```x``` в список слов:

In [14]:
vectorizer.inverse_transform(x)

[array(['car', 'enlighten', 'saw', 'wondering'], dtype='<U79')]

Пропало слово ```I```. Но дело в том, что по умолчанию ```CountVectorizer``` отбрасывает последовательности, короче 2 символов. На это указывает параметр ```token_pattern='(?u)\\b\\w\\w+\\b'```.

Переведем весь набор текстов обучающего датасета в набор векторов, получим матрицу ```X_train```:

In [38]:
X_train = vectorizer.fit_transform(newsgroups_train.data)
X_train.shape

(11314, 12424)

О пользе разреженных матриц. Отношение числа ненулевых элементов ко всем элементам матрицы ```X_train```:

In [39]:
X_train.nnz / np.prod(X_train.shape)

0.004060060810527014


Задача: запустить модель LDA и Gibbs Sampling с числов тегов 20. Вывести топ-10 слов по каждому тегу. Соотнести полученные теги с тегами из датасета, сделать выводы.

* заведем счетчики $n_{k, w}$, $n_{d, k}$, $n_k$, заполненные нулями

In [40]:
import random
x,y = X_train.shape
K = 20
len_dict = len(vectorizer.vocabulary_)
Dict = vectorizer.vocabulary_
n_d_k = np.zeros(shape = (x,K))
n_k_w = np.zeros(shape = (K,len_dict))
n_k = np.zeros(shape = K)
columns = X_train.nonzero()[1]
rows = X_train.nonzero()[0]
non_zero_len= X_train.data.shape[0]
print(n_d_k.shape,n_k_w.shape)

(11314, 20) (20, 12424)


Обновим значения в X_train.data и назначим на их место числа от 1 до 20(номера тега)

* случайным образом расставим теги словам, обновим счетчики $n_{k, w}$, $n_{d, k}$, $n_k$

In [41]:
X_train.data = np.random.randint(low =1 ,high = 21,size = (non_zero_len,))

Обновим наши три счётчика

In [42]:
for i in range(non_zero_len):
    n_k[X_train.data[i]-1] += 1
    n_k_w[X_train.data[i]-1][columns[i]] += 1
    n_d_k[rows[i]][X_train.data[i]-1] += 1

Напишем алгоритм самого блуждания

* пока не сойдемся к стационарному режиму:
  * для каждого $i$ от 1 до $W$:
      * $n_{d_i, z_i} \mathrel{-}= 1$, $n_{z_i, w_i} \mathrel{-}= 1$, $n_{z_i} \mathrel{-}= 1$
      * для каждого $k$ от 1 до $K$:
        * вычисляем $p_k = (n_{d_i, k} + \alpha_k) \frac{n_{k, w_i} + \beta_{w_i}}{n_k + \beta_{sum}}$
      * сэмплим новый $z_i$ из полученного распределения $(p_1, ..., p_K)$
      * $n_{d_i, z_i} \mathrel{+}= 1$, $n_{z_i, w_i} \mathrel{+}= 1$, $n_{z_i} \mathrel{+}= 1$


In [43]:
from tqdm import tqdm
def walk(niter,X_train,n_k,n_k_w,n_d_k,K):
    
    non_zero_len= X_train.data.shape[0]
    columns = X_train.nonzero()[1]
    rows = X_train.nonzero()[0]
    non_zero = X_train.data
    
    a = np.ones(K)
    b = np.ones(X_train.shape[1])
    b_summ = b.sum()
    num = list(np.arange(0,K)+1)
    
    for j in tqdm(range(niter)):
        for i in range(non_zero_len):
                n_k[non_zero[i]-1] -= 1
                n_k_w[non_zero[i]-1][columns[i]] -= 1
                n_d_k[rows[i]][non_zero[i]-1] -= 1
                
                probability = np.array([(n_d_k[rows[i]][c]+a[c])*(n_k_w[c][columns[i]]+b[columns[i]])/(n_k[c]+b_summ)
                                                                                                        for c in range(K)])
                non_zero[i] = int(random.choices(num, weights =  probability/probability.sum())[0])
                
                n_k[non_zero[i]-1] += 1
                n_k_w[non_zero[i]-1][columns[i]] += 1
                n_d_k[rows[i]][non_zero[i]-1] += 1
#         print(f'Номер прошедшей итерации: {j}')
    return non_zero,n_k,n_k_w,n_d_k

In [44]:
rezult,_,n_k_w,_ = walk(50,X_train,n_k,n_k_w,n_d_k,K)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [22:23<00:00, 27.21s/it]


Вывести топ-10 слов по каждому тегу. Соотнести полученные теги с тегами из датасета, сделать выводы.

In [45]:
maximus = np.zeros(shape = (K,10),dtype = int)
for ni in range(10):
    maxis = np.zeros(K,dtype = int)
    for i in range(len_dict):
        maxis = [i if n_k_w[j][i]>n_k_w[j][maxis[j]] else maxis[j] for j in range(K)]
    for j in range(K):
        maximus[j][ni] = maxis[j]
        n_k_w[j][maxis[j]] = 0

In [46]:
for i in range(K):
    dat = np.zeros((1, len_dict))
    for word in maximus[i]:
        dat[0, word] = 1
    print('Топик {}\t\t{}'.format(i,' '.join(vectorizer.inverse_transform(dat)[0])))

Топик 0		actually ago doing got idea read sorry sure tell yes
Топик 1		10 11 12 13 14 15 16 20 25 30
Топик 2		condition edu interested mail offer old power price sale sell
Топик 3		banks case cause edu gordon having lot pitt probably soon
Топик 4		address com edu group information list mail message number send
Топик 5		article end information interested news number post results study university
Топик 6		come day didn going got said saw thought told went
Топик 7		bit card disk drive hard pc software using video windows
Топик 8		02 14 ah ca end hi mr ms ok sp
Топик 9		clinton fact government key law legal private public rights state
Топик 10		cost earth high low nasa project research space using years
Топик 11		better game games league play players season team win year
Топик 12		believe course doesn getting maybe sure tell thought wrong yes
Топик 13		better bike car cars engine going little ll look thing
Топик 14		al com dave edu email internet look phone reply thank
Топик 15		government

Выводы:
а)воспользовавшись советом Ильи, я сжал пороги частотности для слов, таким образом словарь уменьшился в $\approx$ 9 раз.

б) Можем сопоставить 10 главных слов с возможными темами,тогда получится:
  0)  
  
  1)  Циферки (misc.forsale)
  
  2)  Тема про политиков(talk.politics.misc)
  
  3)  
  
  4)  
  
  5)  
  
  6)  Тема про жёсткие диски и тд.(comp.sys.ibm.pc.hardware)
  
  7)  
  
  8)
  
  9)  Тема про космос(sci.space)
  
  10) Тема про игры
  
  11) 
  
  12) Тема про машины и байки
  
  13) Тема про 
  
  14) Тема про войну и убийства (talk.politics.guns)
  
  15) Тема связанная с богом(soc.religion.christian)
  
  16) 
  
  17) 
  
  18) Тема связанная с компьтерами и windows(comp.os.ms-windows.misc)
  
  19) 
  

In [47]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']